# PyTorch 高级教程2：变分自编码器（VAE）

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

print(torch.__version__)

1.12.1


## 硬件配置

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 超参数

In [3]:
# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

## 准备数据

In [4]:
# Create a directory if not exists
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
    
# MNIST dataset
dataset = torchvision.datasets.MNIST(root='../../data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

## 加载数据

In [5]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

## 定义模型：VAE

In [6]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

model = VAE().to(device)

print(model)

## 优化器

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 训练模型

In [8]:
# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # Compute reconstruction loss and kl divergence
        # For KL divergence, see Appendix B in VAE paper or http://yunjey47.tistory.com/43
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

d:\ProgramData\anaconda3\envs\pytorch-tutorial\lib\site-packages\torch\nn\functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
d:\ProgramData\anaconda3\envs\pytorch-tutorial\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/15], Step [10/469], Reconst Loss: 36826.0195, KL Div: 2827.7910
Epoch[1/15], Step [20/469], Reconst Loss: 29603.6289, KL Div: 1227.5696
Epoch[1/15], Step [30/469], Reconst Loss: 27758.5195, KL Div: 1150.4274
Epoch[1/15], Step [40/469], Reconst Loss: 27148.9707, KL Div: 635.3127
Epoch[1/15], Step [50/469], Reconst Loss: 27245.4121, KL Div: 614.3327
Epoch[1/15], Step [60/469], Reconst Loss: 25424.0469, KL Div: 746.2432
Epoch[1/15], Step [70/469], Reconst Loss: 24993.3359, KL Div: 862.5182
Epoch[1/15], Step [80/469], Reconst Loss: 24438.9473, KL Div: 1093.7292
Epoch[1/15], Step [90/469], Reconst Loss: 23707.3750, KL Div: 1179.3423
Epoch[1/15], Step [100/469], Reconst Loss: 23581.5391, KL Div: 1333.9926
Epoch[1/15], Step [110/469], Reconst Loss: 20800.9629, KL Div: 1462.1167
Epoch[1/15], Step [120/469], Reconst Loss: 20831.3105, KL Div: 1697.3671
Epoch[1/15], Step [130/469], Reconst Loss: 19694.0352, KL Div: 1729.7761
Epoch[1/15], Step [140/469], Reconst Loss: 19251.8789, KL Div: 1